# 【實務系列】練習用 Tensorflow 打造一個神經網路！
~~測試失敗~~

參考資料:(https://medium.com/%E7%A8%8B%E5%BC%8F%E6%84%9B%E5%A5%BD%E8%80%85/%E5%AF%A6%E5%8B%99%E7%B3%BB%E5%88%97-%E7%B7%B4%E7%BF%92%E7%94%A8-tensorflow-%E6%89%93%E9%80%A0%E4%B8%80%E5%80%8B%E7%A5%9E%E7%B6%93%E7%B6%B2%E8%B7%AF-9ca637c176a5)

### 第一步：匯入套件跟資料集

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 第二步：參數設定

In [2]:
epochs = 10
batch_size = 100
x_train = x_train / 255.0 # scaler to 0~1
x_test = x_test / 255.0 # scaler to 0~1
x_test = tf.Variable(x_test)

### 第三步：切分 batch

In [3]:
def get_batch(x_data, y_data, batch_size):
    idxs = np.random.randint(0, len(y_data), batch_size)
    return x_data[idxs, :, :], y_data[idxs]

x_data, y_data = get_batch(x_train, y_train, batch_size=100)
print(y_data.shape)
print(x_data.shape)

(100,)
(100, 28, 28)


### 第四步：打造神經網路

In [4]:
w1 = tf.Variable(tf.random.normal([784, 300], stddev=0.03, name='w1'))
b1 = tf.Variable(tf.random.normal([300]), name='b1')
w2 = tf.Variable(tf.random.normal([300, 10], stddev=0.03, name='w2'))
b2 = tf.Variable(tf.random.normal([300]), name='b2')

In [5]:
def nn_model(x_input, W1, b1 ,W2, b2):
    x_input = tf.reshape(x_input, shape=(x_input.shape[0], -1))
    # y = (Wx + b) * activation function
    x = tf.add(tf.matmul(tf.cast(x_input, tf.float32), W1), b1)
    x = tf.nn.relu(x)
    outputs = tf.add(tf.matmul(x, W2), b2)
    outputs = tf.nn.softmax(outputs)
    return outputs

#### Keras

In [6]:
# input_layer = Input(shape=(784,), name='input_layer')
# dense = Dense(300, activation='relu', name='hidden_layer')(input_layer)
# output_layer = Dense(10, activation='softmax', name='output_layer')(dense)
# model_ = Model(inputs=input_layer, outputs=output_layer)
# model_.summary()

### 第五步：定義 loss function

In [7]:
def loss_fn(outputs, labels):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=outputs))
    return cross_entropy

### 第六步：定義 optimizer

In [8]:
optimizer = tf.keras.optimizers.Adam()

### 第七步：訓練網路

In [9]:
total_batch = int(len(y_train) / batch_size)
for epoch in range(epochs):
    avg_loss = 0
    for i in range(total_batch):
        batch_x, batch_y = get_batch(x_train, y_train, batch_size=batch_size)
        batch_x = tf.Variable(batch_x)
        # batch_y.shape = (100, )
        batch_y = tf.Variable(batch_y)
        batch_y = tf.one_hot(batch_y, 10)
        with tf.GradientTape() as tape:
            pred = nn_model(batch_x, w1, b1, w2, b2)
            loss = loss_fn(outputs=pred, labels=batch_y)
        
        gradients = tape.gradient(loss, [w1, b1, w2, b2])
        optimizer.apply_gradients(zip(gradients, [w1, b1, w2, b2]))
        avg_loss += loss / total_batch
    
    # validate training data
    train_pred = nn_model(x_train, w1, b1, w2, b2)
    train_max_idxs = tf.argmax(train_pred, axis=1)
    acc = np.sum(train_max_idxs.numpy() == y_train) / len(y_train)
    
    # validate testing data
    test_pred = nn_model(x_test, w1, b1, w2, b2)
    y_test_one_hot = tf.one_hot(y_test, 10)
    test_loss = loss_fn(outputs=test_pred, labels=y_test_one_hot)
    max_idxs = tf.argmax(test_pred, axis=1)
    test_acc = np.sum(max_idxs.numpy() == y_test) / len(y_test)
    print(f"Epoch: {epoch+1}, loss={avg_loss:.3f}, , acc={acc:.3f}, val_loss={test_loss:.3f}, val_acc:{test_acc*100:.3f}")

print("Training complete")

InvalidArgumentError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [100,10] vs. [300] [Op:AddV2]